## Building a UniFHy Model

Walkthrough of how to build a UniFHy model, and the common stumbling blocks

In [1]:
%load_ext autoreload
%autoreload 2

# Ensure we're not picking up the jaspy version of ESMF if running on JASMIN
import os
try:
    os.environ.pop("ESMFMKFILE")
except KeyError:
    pass

import cf
import os
from datetime import datetime, timedelta
#import matplotlib.pyplot as plt
#import cartopy as cp
from unifhy.settings import rtol, atol
rtol_ = rtol()
atol_ = atol()

import unifhy
import unifhycontrib.artemis
import unifhycontrib.rfm
import unifhycontrib.ltls

/home/users/mattjbr/miniconda3/envs/unifhy2025/lib/python3.14/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


In [2]:
unifhy.__version__

'1.0.0'

In [3]:
cf.__version__

'3.18.2'

## Setting the scene

View the [diagram of possible model Components](https://unifhy-org.github.io/unifhy/index.html) to get a feel for the framework and decide which parts of the hydrological cycle you will model. Models representing these parts of the hydrological cycle can be found in the [unifhy-org github organisation](https://github.com/unifhy-org). Those you don't wish to model use can be represented with data files (and thus a [Data Component](https://unifhy-org.github.io/unifhy/api/classes/unifhy.DataComponent.html)) or ignored entirely (a [Null Component](https://unifhy-org.github.io/unifhy/api/classes/unifhy.NullComponent.html)). A full [tutorial](https://unifhy-org.github.io/unifhy/users/tutorial.html) is available, which this notebook will largely follow.

Here we will assemble a Model made up of:
- Surface: [ARTEMIS model](https://github.com/unifhy-org/unifhycontrib-artemis)
- SubSurface: [ARTEMIS model](https://github.com/unifhy-org/unifhycontrib-artemis)
- Open Water: [RFM model](https://github.com/unifhy-org/unifhycontrib-rfm)
- Nutrients Surface: [Null Component](https://unifhy-org.github.io/unifhy/api/classes/unifhy.NullComponent.html)
- Nutrients SubSurface: [Data Component](https://unifhy-org.github.io/unifhy/api/classes/unifhy.DataComponent.html)
- Nutrients Open Water: [LTLS Components](https://github.com/unifhy-org/unifhycontrib-ltls/tree/dev) (note: currently in private beta so you need to be granted access)

Some UniFHy specific terminology it's useful to know before continuing:
- in*wards*/out*wards*: These are the transfers between Components, that UniFHy manages. Represented by the pink lines in the above diagram.
- in*puts*: Data coming from outside the framework in the form of files, needed for a given Component to run. Can be though of as the 'driving data', e.g. atmosphere data for the SurfaceLayer Component.
- states: The states correspond to the component variables that need to be given initial values to start the component time integration, and whose values need to be sustained from one time step to the next. They are typically internal to a given Component, as part of it's calculations.
- out*puts*: Any variable that is not one of the above, that you would like to make 'recordable' (outputtable).
- records: What UniFHy uses to refer to variables flagged for output to file.
- parameters: The parameters are those variables in a Component that are subject to tuning/calibration
- constants: Treated by UniFHy the same as parameters. The difference is purely semantic, to flag that these variables are not likely to be subject to tuning/calibration.

For a full, rigorous definition of these, see the [contributor guide](https://unifhy-org.github.io/unifhy/contributors/development.html) section of the documentation.

### Getting access to the LTLS model for the NutrientOpenWater Component

Until this is made public, access is granted on a case-by-case basis by making you a member of the containing GitHub organisation. Once access is granted, the package needs to be installed into the python environment where you are running UniFHy. 

The way I typically do this is to authenticate with an SSH-agent in a terminal following [GitHub's instructions](https://docs.github.com/en/authentication/connecting-to-github-with-ssh/generating-a-new-ssh-key-and-adding-it-to-the-ssh-agent), then install using pip:
```
pip install git+ssh://git@github.com/unifhy-org/unifhycontrib-ltls.git@splitapart
```

## Building the Model

To give you an idea of what we're working towards, the final model assembly command looks like:

```
model = unifhy.Model(
    identifier='tutorial',
    config_directory='configurations',
    saving_directory='outputs',
    surfacelayer=unifhycontrib.artemis.SurfaceLayerComponent(
        'outputs', timedomain, spacedomain, dataset_surfacelayer,
        parameters={}
    ),
    subsurface=unifhycontrib.artemis.SubSurfaceComponent(
        'outputs', timedomain, spacedomain, dataset_subsurface,
        parameters={}
    ),
    openwater=unifhycontrib.rfm.OpenWaterComponent(
        'outputs', timedomain, spacedomain, dataset_openwater,
        parameters={'c_land': (0.20, 'm s-1'),
                    'cb_land': (0.10, 'm s-1'),
                    'c_river': (0.62, 'm s-1'),
                    'cb_river': (0.15, 'm s-1'),
                    'ret_l': (0.0, '1'),
                    'ret_r': (0.005, '1'),
                    'routing_length': (50000, 'm')},
        records={
            'outgoing_water_volume_transport_along_river_channel': {
                timedelta(days=1): ['mean']
            }
        }
    ),
    nutrientsurfacecomponent = unifhy.NullComponent(
         timedomain, spacedomain, unifhy.component.NutrientSurfaceLayerComponent
    ),
    nutrientsubsurface=unifhy.DataComponent(
         timedomain, spacedomain, dataset_nutrientsubsurface,
         unifhy.component.NutrientSubSurfaceComponent
    ),
    nutrientopenwater=unifhycontrib.ltls.NutrientOpenWaterComponent(
        'outputs', timedomain, spacedomain, dataset_nutrientopenwater,
        parameters={}
    )
)
```

The key bits to focus on are what is specified for surfacelayer, subsurface, openwater, nutrientsurfacecomponent, nutrientsubsurface, and nutrientopenwater These are the Components that make up the model.

We can see that all the Components need a timedomain and a spacedomain, specifying details about the time and space domains each Component will use. There are [rules](https://unifhy-org.github.io/unifhy/users/tutorial.html#framework) about how different these can be for different Components. 

Here we will let all the Components use the same [TimeDomain](https://unifhy-org.github.io/unifhy/api/classes/unifhy.TimeDomain.html#unifhy.TimeDomain) and [SpaceDomain](https://unifhy-org.github.io/unifhy/api/classes/unifhy.LatLonGrid.html) for simplicity. These are constructed as follows:

In [4]:
timedomain = unifhy.TimeDomain.from_start_end_step(
   start=datetime(2017, 1, 1, 0, 0, 0),
   end=datetime(2017, 2, 1, 0, 0, 0),
   step=timedelta(hours=1),
   calendar='gregorian'
)

print(timedomain)

TimeDomain(
    time (744,): [2017-01-01 00:00:00, ..., 2017-01-31 23:00:00] gregorian
    bounds (744, 2): [[2017-01-01 00:00:00, ..., 2017-02-01 00:00:00]] gregorian
    calendar: gregorian
    units: seconds since 1970-01-01 00:00:00Z
    period: 31 days, 0:00:00
    timedelta: 1:00:00
)


In [5]:
spacedomain = unifhy.LatLonGrid.from_extent_and_resolution(
   latitude_extent=(51, 55),
   latitude_resolution=0.5,
   longitude_extent=(-2, 1),
   longitude_resolution=0.5
)

print(spacedomain)

LatLonGrid(
    shape {Y, X}: (8, 6)
    Y, latitude (8,): [51.25, ..., 54.75] degrees_north
    X, longitude (6,): [-1.75, ..., 0.75] degrees_east
    Y_bounds (8, 2): [[51.0, ..., 55.0]] degrees_north
    X_bounds (6, 2): [[-2.0, ..., 1.0]] degrees_east
)


Three additional properties of SpaceDomain may require to be set depending on a Component’s requirements: land_sea_mask, flow_direction, and cell_area:

- land_sea_mask may be used by a component to be aware of where there is land and where there is sea, but if set, it is also used to mask the component records;
- flow_direction may be used by a component to determine where to move flow downstream, it is namely compulsory if the component is using the SpaceDomain’s route method;
 - cell_area may be used by a component to determine the surface area of each spatial element in the component spacedomain. For some spacedomains (e.g. LatLonGrid, RotatedLatLonGrid, BritishNationalGrid), the cell area is calculated automatically by the framework if not already set.


To check whether a component needs any of these spatial properties, each Component can be queried via its methods ```requires_land_sea_mask()```, ```requires_flow_direction()```, and ```requires_cell_area()```.

In [6]:
unifhycontrib.rfm.OpenWaterComponent.requires_flow_direction()

True

Given there's at least one Component that requires flow directions, we need to read these in and assign these to the SpaceDomain that the Component that requires the flow directions will run on. The typical way of doing this is to read the directions in from a file using the cf python library that UniFHy uses under the hood.

In [7]:
spacedomain.flow_direction = (
    cf.read('development/unifhy/data/flow_direction.nc').select_field("long_name=flow direction")
)

{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}


The same goes for the land sea mask

In [8]:
spacedomain.land_sea_mask = (
    cf.read('development/unifhy/data/land_sea_mask.nc').select_field('land_binary_mask')
)

{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}


Looking again at the final model build command, we can see that all Components bar the Null Component have a dataset input:

```
model = unifhy.Model(
    identifier='tutorial',
    config_directory='configurations',
    saving_directory='outputs',
    surfacelayer=unifhycontrib.artemis.SurfaceLayerComponent(
        'outputs', timedomain, spacedomain, dataset_surfacelayer,
        parameters={}
    ),
    subsurface=unifhycontrib.artemis.SubSurfaceComponent(
        'outputs', timedomain, spacedomain, dataset_subsurface,
        parameters={}
    ),
    openwater=unifhycontrib.rfm.OpenWaterComponent(
        'outputs', timedomain, spacedomain, dataset_openwater,
        parameters={'c_land': (0.20, 'm s-1'),
                    'cb_land': (0.10, 'm s-1'),
                    'c_river': (0.62, 'm s-1'),
                    'cb_river': (0.15, 'm s-1'),
                    'ret_l': (0.0, '1'),
                    'ret_r': (0.005, '1'),
                    'routing_length': (50000, 'm')},
        records={
            'outgoing_water_volume_transport_along_river_channel': {
                timedelta(days=1): ['mean']
            }
        }
    ),
    nutrientsurfacecomponent = unifhy.NullComponent(
         timedomain, spacedomain, unifhy.component.NutrientSurfaceLayerComponent
    ),
    nutrientsubsurface=unifhy.DataComponent(
         timedomain, spacedomain, dataset_nutrientsubsurface,
         unifhy.component.NutrientSubSurfaceComponent
    ),
    nutrientopenwater=unifhycontrib.ltls.NutrientOpenWaterComponent(
        'outputs', timedomain, spacedomain, dataset_nutrientopenwater,
        parameters={}
    )
)
```

These Dataset objects collect together the data file inputs needed to drive each Component. To find out what inputs are required for each Component use it's ```_inputs_info``` attribute.

Surface:

In [9]:
unifhycontrib.artemis.SurfaceLayerComponent._inputs_info

{'precipitation_flux': {'units': 'kg m-2 s-1', 'kind': 'dynamic'},
 'specific_humidity': {'units': '1', 'kind': 'dynamic'},
 'surface_downwelling_shortwave_flux_in_air': {'units': 'W m-2',
  'kind': 'dynamic'},
 'surface_downwelling_longwave_flux_in_air': {'units': 'W m-2',
  'kind': 'dynamic'},
 'air_temperature': {'units': 'K', 'kind': 'dynamic'},
 'surface_albedo': {'units': '1', 'kind': 'static'},
 'wind_speed': {'units': 'm s-1', 'kind': 'dynamic'},
 'vegetation_height': {'units': 'm', 'kind': 'static'},
 'leaf_area_index': {'units': '1',
  'kind': 'climatologic',
  'frequency': 'monthly'}}

In [10]:
dataset_surfacelayer = unifhy.DataSet(
    files=['development/unifhy/data/surface_downwelling_longwave_flux_in_air.nc',
           'development/unifhy/data/surface_downwelling_shortwave_flux_in_air.nc',
           'development/unifhy/data/specific_humidity.nc',
           'development/unifhy/data/air_temperature.nc',
           'development/unifhy/data/wind_speed.nc',
           'development/unifhy/data/precipitation_flux.nc',
           'development/unifhy/data/leaf_area_index.nc',
           'development/unifhy/data/canopy_height.nc',
           'development/unifhy/data/soil_albedo.nc'],
    name_mapping={'leaf-area index': 'leaf_area_index',
                  'canopy height': 'vegetation_height',
                  'soil albedo': 'surface_albedo'}
)

print(dataset_surfacelayer)

DataSet{
    air_temperature(time(744), latitude(20), longitude(28)) K
    leaf_area_index(time(12), latitude(360), longitude(720)) 1
    precipitation_flux(time(744), latitude(20), longitude(28)) kg m-2 s-1
    specific_humidity(time(744), latitude(20), longitude(28)) kg kg-1
    surface_albedo(latitude(360), longitude(720)) 1
    surface_downwelling_longwave_flux_in_air(time(744), latitude(20), longitude(28)) W m-2
    surface_downwelling_shortwave_flux_in_air(time(744), latitude(20), longitude(28)) W m-2
    vegetation_height(latitude(360), longitude(720)) m
    wind_speed(time(744), latitude(20), longitude(28)) m s-1
}


SubSurface:

In [11]:
unifhycontrib.artemis.SubSurfaceComponent._inputs_info

{'topmodel_saturation_capacity': {'units': 'mm m-1', 'kind': 'static'},
 'saturated_hydraulic_conductivity': {'units': 'm s-1', 'kind': 'static'},
 'topographic_index': {'units': '1', 'kind': 'static'}}

In [12]:
dataset_subsurface = unifhy.DataSet(
    files=['development/unifhy/data/saturated_hydraulic_conductivity.nc',
           'development/unifhy/data/available_water_storage_capacity.nc',
           'development/unifhy/data/topographic_index.nc'],
    name_mapping={
        'saturated hydraulic conductivity': 'saturated_hydraulic_conductivity',
        'available water storage capacity': 'topmodel_saturation_capacity',
        'topographic index': 'topographic_index'
     }
)

OpenWater:

In [13]:
unifhycontrib.rfm.OpenWaterComponent._inputs_info

{'flow_accumulation': {'units': '1',
  'kind': 'static',
  'description': 'drainage area (specified in number of cells) draining to a grid box'}}

In [14]:
dataset_openwater = unifhy.DataSet(
    files='development/unifhy/data/flow_accumulation.nc',
    name_mapping={
        'RFM drainage area in cell counts (WFDEI)': 'flow_accumulation'
    }
)

Nutrients OpenWater:

In [15]:
unifhycontrib.ltls.NutrientOpenWaterComponent._inputs_info

{}

No input datasets needed.

Nutrients SubSurface:

For DataComponents, we need to replace all the *outward transfers* of the Component we are subtituting the DataComponent for. Here, this is the NutrientSubSurface Component, which has the following outward transfers:

In [16]:
unifhy.NutrientSubSurfaceComponent._outwards_info

{'mass_flux_of_dissolved_inorganic_carbon_from_soil_in_surface_runoff': {'units': 'kg m-2 s-1',
  'to': ['nutrientopenwater'],
  'method': 'mean'},
 'mass_flux_of_dissolved_organic_carbon_from_soil_in_surface_runoff': {'units': 'kg m-2 s-1',
  'to': ['nutrientopenwater'],
  'method': 'mean'},
 'mass_flux_of_dissolved_nitrogen_as_ammonium_from_soil_in_surface_runoff': {'units': 'kg m-2 s-1',
  'to': ['nutrientopenwater'],
  'method': 'mean'},
 'mass_flux_of_dissolved_nitrogen_as_nitrate_from_soil_in_surface_runoff': {'units': 'kg m-2 s-1',
  'to': ['nutrientopenwater'],
  'method': 'mean'},
 'mass_flux_of_dissolved_organic_nitrogen_from_soil_in_surface_runoff': {'units': 'kg m-2 s-1',
  'to': ['nutrientopenwater'],
  'method': 'mean'},
 'mass_flux_of_dissolved_phosphorus_from_soil_in_surface_runoff': {'units': 'kg m-2 s-1',
  'to': ['nutrientopenwater'],
  'method': 'mean'},
 'mass_flux_of_dissolved_calcium_from_soil_in_surface_runoff': {'units': 'kg m-2 s-1',
  'to': ['nutrientopenwate

Technically, not all of these are required *if* the receiving Component isn't requiring them. Which inwards transfers that are required by a Component can be checked as follows:

In [17]:
unifhycontrib.ltls.NutrientOpenWaterComponent._inwards

{'mass_concentration_of_carbon_dioxide_in_air',
 'mass_flux_of_dissolved_calcium_from_soil_in_subsurface_runoff',
 'mass_flux_of_dissolved_calcium_from_soil_in_surface_runoff',
 'mass_flux_of_dissolved_inorganic_carbon_from_soil_in_subsurface_runoff',
 'mass_flux_of_dissolved_inorganic_carbon_from_soil_in_surface_runoff',
 'mass_flux_of_dissolved_nitrogen_as_ammonium_from_soil_in_subsurface_runoff',
 'mass_flux_of_dissolved_nitrogen_as_ammonium_from_soil_in_surface_runoff',
 'mass_flux_of_dissolved_nitrogen_as_nitrate_from_soil_in_subsurface_runoff',
 'mass_flux_of_dissolved_nitrogen_as_nitrate_from_soil_in_surface_runoff',
 'mass_flux_of_dissolved_organic_carbon_from_soil_in_subsurface_runoff',
 'mass_flux_of_dissolved_organic_carbon_from_soil_in_surface_runoff',
 'mass_flux_of_dissolved_organic_nitrogen_from_soil_in_subsurface_runoff',
 'mass_flux_of_dissolved_organic_nitrogen_from_soil_in_surface_runoff',
 'mass_flux_of_dissolved_phosphorus_from_soil_in_subsurface_runoff',
 'mass_fl

From which we can see that all the outward transfers from NutrientSubSurface to NutrientOpenWater are required and we can load into a dataset:

In [18]:
dataset_nutrientsubsurface = unifhy.DataSet(
    files=['development/unifhy/data/nutrients_runoffs/DOC_ro.nc',
           'development/unifhy/data/nutrients_runoffs/DIC_ro.nc',
           'development/unifhy/data/nutrients_runoffs/DON_ro.nc',
           'development/unifhy/data/nutrients_runoffs/Ammonium_ro.nc',
           'development/unifhy/data/nutrients_runoffs/Nitrate_ro.nc',
           'development/unifhy/data/nutrients_runoffs/TDP_ro.nc',
           'development/unifhy/data/nutrients_runoffs/Calcium_ro.nc',
           'development/unifhy/data/nutrients_runoffs/Sulphate_ro.nc',
           'development/unifhy/data/nutrients_runoffs/Silicon_ro.nc',
           'development/unifhy/data/nutrients_runoffs/DOC_gw.nc',
           'development/unifhy/data/nutrients_runoffs/DIC_gw.nc',
           'development/unifhy/data/nutrients_runoffs/DON_gw.nc',
           'development/unifhy/data/nutrients_runoffs/Ammonium_gw.nc',
           'development/unifhy/data/nutrients_runoffs/Nitrate_gw.nc',
           'development/unifhy/data/nutrients_runoffs/TDP_gw.nc',
           'development/unifhy/data/nutrients_runoffs/Calcium_gw.nc',
           'development/unifhy/data/nutrients_runoffs/Sulphate_gw.nc',
           'development/unifhy/data/nutrients_runoffs/Silicon_gw.nc'
           ])

Nutrients Surface Layer:

Surface component possible outwards (to other components) transfers. We will use a Null component for this meaning all these outwards transfers are provided as nulls/NaNs/zeros.

In [19]:
unifhy.SurfaceLayerComponent._outwards_info

{'canopy_liquid_throughfall_and_snow_melt_flux': {'units': 'kg m-2 s-1',
  'to': ['subsurface'],
  'method': 'mean'},
 'transpiration_flux_from_root_uptake': {'units': 'kg m-2 s-1',
  'to': ['subsurface'],
  'method': 'mean'},
 'direct_water_evaporation_flux_from_soil': {'units': 'kg m-2 s-1',
  'to': ['subsurface'],
  'method': 'mean'},
 'water_evaporation_flux_from_standing_water': {'units': 'kg m-2 s-1',
  'to': ['subsurface'],
  'method': 'mean'},
 'water_evaporation_flux_from_open_water': {'units': 'kg m-2 s-1',
  'to': ['openwater'],
  'method': 'mean'},
 'direct_throughfall_flux': {'units': 'kg m-2 s-1',
  'to': ['openwater'],
  'method': 'mean'}}

We are almost ready to build the model!

Looking once more at the command we'll use to build the model:

```
model = unifhy.Model(
    identifier='tutorial',
    config_directory='configurations',
    saving_directory='outputs',
    surfacelayer=unifhycontrib.artemis.SurfaceLayerComponent(
        'outputs', timedomain, spacedomain, dataset_surfacelayer,
        parameters={}
    ),
    subsurface=unifhycontrib.artemis.SubSurfaceComponent(
        'outputs', timedomain, spacedomain, dataset_subsurface,
        parameters={}
    ),
    openwater=unifhycontrib.rfm.OpenWaterComponent(
        'outputs', timedomain, spacedomain, dataset_openwater,
        parameters={'c_land': (0.20, 'm s-1'),
                    'cb_land': (0.10, 'm s-1'),
                    'c_river': (0.62, 'm s-1'),
                    'cb_river': (0.15, 'm s-1'),
                    'ret_l': (0.0, '1'),
                    'ret_r': (0.005, '1'),
                    'routing_length': (50000, 'm')},
        records={
            'outgoing_water_volume_transport_along_river_channel': {
                timedelta(days=1): ['mean']
            }
        }
    ),
    nutrientsurfacecomponent = unifhy.NullComponent(
         timedomain, spacedomain, unifhy.component.NutrientSurfaceLayerComponent
    ),
    nutrientsubsurface=unifhy.DataComponent(
         timedomain, spacedomain, dataset_nutrientsubsurface,
         unifhy.component.NutrientSubSurfaceComponent
    ),
    nutrientopenwater=unifhycontrib.ltls.NutrientOpenWaterComponent(
        'outputs', timedomain, spacedomain, dataset_nutrientopenwater,
        parameters={}
    )
)
```

The only thing we've yet to think about is the parameters that we can see going in to the OpenWater Component. To see if a Component we're using in our model requires any parameters we do the following:

In [20]:
unifhycontrib.rfm.OpenWaterComponent._parameters_info

{'c_land': {'units': 'm s-1',
  'description': 'kinematic wave speed for surface flow in a land grid box on the river routing grid'},
 'cb_land': {'units': 'm s-1',
  'description': 'kinematic wave speed for subsurface flow in a land grid box on the river routing grid'},
 'c_river': {'units': 'm s-1',
  'description': 'kinematic wave speed for surface flow in a river grid box on the river routing grid'},
 'cb_river': {'units': 'm s-1',
  'description': 'kinematic wave speed for subsurface flow in a river grid box on the river routing grid'},
 'ret_l': {'units': '1',
  'description': 'land return flow fraction (resolution dependent)'},
 'ret_r': {'units': '1',
  'description': 'river return flow fraction (resolution dependent)'},
 'routing_length': {'units': 'm', 'description': 'length of the routing path'}}

Ah, but what about the 'records' parameter? You're right, I forgot that. With this parameter you can describe what variables you would like recorded to an output file. The variables can be any of the transfers (in*wards* or out*wards* variables) between the Components, Component out*puts* or 'states', which are essentially any variable that is assigned to and/or used during the integration/time step of a given Component, that isn't already an inward/outward/input/output variable. 

With all that, we are finally ready to build the model!

In [21]:
def build_model(surfacelayer, subsurface, openwater, nutrientsurfacelayer, nutrientsubsurface, nutrientopenwater):
    model = unifhy.Model(
        identifier = 'tutorial',
        config_directory = '.',
        saving_directory = 'outputs',
        surfacelayer = surfacelayer,
        subsurface = subsurface,
        openwater = openwater,
        nutrientsurfacelayer = nutrientsurfacelayer,
        nutrientsubsurface = nutrientsubsurface,
        nutrientopenwater=nutrientopenwater
    )
    return model

In [22]:
surfacelayer=unifhycontrib.artemis.SurfaceLayerComponent(
            'outputs', timedomain, spacedomain, dataset_surfacelayer,
            parameters={})
subsurface=unifhycontrib.artemis.SubSurfaceComponent(
            'outputs', timedomain, spacedomain, dataset_subsurface,
            parameters={})
openwater=unifhycontrib.rfm.OpenWaterComponent(
            'outputs', timedomain, spacedomain, dataset_openwater,
            parameters={'c_land': (0.20, 'm s-1'),
                        'cb_land': (0.10, 'm s-1'),
                        'c_river': (0.62, 'm s-1'),
                        'cb_river': (0.15, 'm s-1'),
                        'ret_l': (0.0, '1'),
                        'ret_r': (0.005, '1'),
                        'routing_length': (50000, 'm')},
            records={
                'outgoing_water_volume_transport_along_river_channel': {
                    timedelta(days=1): ['mean']
                }
            }
        )
nutrientsurfacelayer = unifhy.NullComponent(
             timedomain, spacedomain, unifhy.component.NutrientSurfaceLayerComponent)
nutrientsubsurface=unifhy.DataComponent(
             timedomain, spacedomain, dataset_nutrientsubsurface,
             unifhy.component.NutrientSubSurfaceComponent)
nutrientopenwater=unifhycontrib.ltls.NutrientOpenWaterComponent(
            'outputs', timedomain, spacedomain,
            parameters={})

model = build_model(surfacelayer, subsurface, openwater, nutrientsurfacelayer, nutrientsubsurface, nutrientopenwater)

{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}
{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}
{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}
{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}
{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}
{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}
{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}
{'longitude': <CF Query: (w

Success!

### Now let's break things...

So far this has all worked well because the input data files we've been using have been specifically formatted to work with UniFHy, which is an extra, fiddly, step that users have to go through and something we want to avoid. Ideally we want users to be able to load a dataset from the EIDC or other data centre straight into UniFHy without having to do any further data wrangling. 

So let's try that with the meteorological driving data that is the input data for the SurfaceLayer Component. We'll use CHESS-MET data, a classic dataset used for hydrological modelling, that we would want to work out of the box with UniFHy, and keep everything else the same as the working version above. 

**Problem 1 (Major): Can't mix and match Component grid types**

The first problem we run into is that CHESS-MET is on the OSGB x/y cartesian grid instead of a lat/lon grid. All the Components within a Model must run on the same Grid Type (OSGB, LatLon or RotatedPole currently), and all the data files in a Dataset must be on the same Grid Type too. Therefore we cannot mix and match the lon/lat data we've so far used with the x/y data of CHESS-MET, so straight away we have to find alternative data sources for all the input data files if we want to use CHESS-MET.

In [23]:
dataset_surfacelayer = unifhy.DataSet(
    files=['development/unifhy/data/surface_downwelling_longwave_flux_in_air.nc',
           'development/unifhy/data/surface_downwelling_shortwave_flux_in_air.nc',
           'development/unifhy/data/specific_humidity.nc',
           'unifhy_driving_files/uk/chess-met-unedited/chess-met_tas_gb_1km_daily_20170101-20170131.nc',
           'development/unifhy/data/wind_speed.nc',
           'development/unifhy/data/precipitation_flux.nc',
           'development/unifhy/data/leaf_area_index.nc',
           'development/unifhy/data/canopy_height.nc',
           'development/unifhy/data/soil_albedo.nc'],
    name_mapping={'leaf-area index': 'leaf_area_index',
                  'canopy height': 'vegetation_height',
                  'soil albedo': 'surface_albedo',
                  'Near-surface air temperature': 'air_temperature'}
)

print(dataset_surfacelayer)

DataSet{
    air_temperature(time(31), projection_y_coordinate(1057), projection_x_coordinate(656)) K
    leaf_area_index(time(12), latitude(360), longitude(720)) 1
    precipitation_flux(time(744), latitude(20), longitude(28)) kg m-2 s-1
    specific_humidity(time(744), latitude(20), longitude(28)) kg kg-1
    surface_albedo(latitude(360), longitude(720)) 1
    surface_downwelling_longwave_flux_in_air(time(744), latitude(20), longitude(28)) W m-2
    surface_downwelling_shortwave_flux_in_air(time(744), latitude(20), longitude(28)) W m-2
    vegetation_height(latitude(360), longitude(720)) m
    wind_speed(time(744), latitude(20), longitude(28)) m s-1
}


It reads in the dataset fine, but...

In [24]:
surfacelayer=unifhycontrib.artemis.SurfaceLayerComponent(
        'outputs', timedomain, spacedomain, dataset_surfacelayer,
        parameters={})

model = build_model(surfacelayer, subsurface, openwater, nutrientsurfacelayer, nutrientsubsurface, nutrientopenwater)

{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}
{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}
{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}
{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}


ValueError: No dimension coordinate constructs found with identity 'longitude', RuntimeError('field not compatible with LatLonGrid')

...fails when trying to build the model. The error message doesn't explain which field/variable it is that is incompatible, though.

So let's try with some other data that *is* on a latlon grid, such as ERA5.

**Problem 2 (Minor): Unintuitive name mapping & name handling**

UniFHy expects the variable names in the datafiles in the DataSets to match the the names specified in the ```_inputs_info``` of the Component. So far, so fair enough. However, because of the enforced CF-Conventions UniFHy applies, the name UniFHy checks is the *long_name* or *standard_name* attributes of the netcdf variable, not it's actual name, causing some confusing error where it complains a certain variable doesn't exist in the DataSet. 

UniFHy does provide a name-mapping argument in the DataSet designed to prevent mass re-names of the NetCDF file variables themselves and instead provide the name of the variable in the NetCDF file and say which variable in the ```_inputs_info``` it matches to. Again however, the variable name that has to be specified is the *standard_name* or *long_name* attributes, not what you would think of as the actual name of the variable. This behaviour is demonstrated below:

In [25]:
unifhycontrib.artemis.SurfaceLayerComponent._inputs_info

{'precipitation_flux': {'units': 'kg m-2 s-1', 'kind': 'dynamic'},
 'specific_humidity': {'units': '1', 'kind': 'dynamic'},
 'surface_downwelling_shortwave_flux_in_air': {'units': 'W m-2',
  'kind': 'dynamic'},
 'surface_downwelling_longwave_flux_in_air': {'units': 'W m-2',
  'kind': 'dynamic'},
 'air_temperature': {'units': 'K', 'kind': 'dynamic'},
 'surface_albedo': {'units': '1', 'kind': 'static'},
 'wind_speed': {'units': 'm s-1', 'kind': 'dynamic'},
 'vegetation_height': {'units': 'm', 'kind': 'static'},
 'leaf_area_index': {'units': '1',
  'kind': 'climatologic',
  'frequency': 'monthly'}}

We can see from the SurfaceLayerComponent's ```_inputs_info``` that we need to map the temperature variable name to *air_temperature*. 

In [26]:
!ncdump -h unifhy_driving_files/era5-unedited/2m_temperature_2017.nc

netcdf \2m_temperature_2017 {
dimensions:
	time = UNLIMITED ; // (8760 currently)
	longitude = 44 ;
	latitude = 47 ;
variables:
	int time(time) ;
		time:standard_name = "time" ;
		time:long_name = "time" ;
		time:units = "hours since 1900-01-01 00:00:00.0" ;
		time:calendar = "gregorian" ;
		time:axis = "T" ;
	float longitude(longitude) ;
		longitude:standard_name = "longitude" ;
		longitude:long_name = "longitude" ;
		longitude:units = "degrees_east" ;
		longitude:axis = "X" ;
	float latitude(latitude) ;
		latitude:standard_name = "latitude" ;
		latitude:long_name = "latitude" ;
		latitude:units = "degrees_north" ;
		latitude:axis = "Y" ;
	double t2m(time, latitude, longitude) ;
		t2m:long_name = "2 metre temperature" ;
		t2m:units = "K" ;
		t2m:_FillValue = -32767. ;
		t2m:missing_value = -32767. ;

// global attributes:
		:CDI = "Climate Data Interface version 1.9.8 (https://mpimet.mpg.de/cdi)" ;
		:Conventions = "CF-1.6" ;
		:history = "Fri Mar 13 07:14:00 2020: cdo -O -b F64 -sell

And we can see that the temperature variable in the netcdf file is 't2m'

In [27]:
dataset_surfacelayer = unifhy.DataSet(
    files=['development/unifhy/data/surface_downwelling_longwave_flux_in_air.nc',
           'development/unifhy/data/surface_downwelling_shortwave_flux_in_air.nc',
           'development/unifhy/data/specific_humidity.nc',
           'unifhy_driving_files/era5-unedited/2m_temperature_2017.nc',
           'development/unifhy/data/wind_speed.nc',
           'development/unifhy/data/precipitation_flux.nc',
           'development/unifhy/data/leaf_area_index.nc',
           'development/unifhy/data/canopy_height.nc',
           'development/unifhy/data/soil_albedo.nc'],
    name_mapping={'leaf-area index': 'leaf_area_index',
                  'canopy height': 'vegetation_height',
                  'soil albedo': 'surface_albedo',
                  't2m': 'air_temperature'} # do the name mapping here
)

print(dataset_surfacelayer)

DataSet{
    2 metre temperature(time(8760), latitude(47), longitude(44)) K
    leaf_area_index(time(12), latitude(360), longitude(720)) 1
    precipitation_flux(time(744), latitude(20), longitude(28)) kg m-2 s-1
    specific_humidity(time(744), latitude(20), longitude(28)) kg kg-1
    surface_albedo(latitude(360), longitude(720)) 1
    surface_downwelling_longwave_flux_in_air(time(744), latitude(20), longitude(28)) W m-2
    surface_downwelling_shortwave_flux_in_air(time(744), latitude(20), longitude(28)) W m-2
    vegetation_height(latitude(360), longitude(720)) m
    wind_speed(time(744), latitude(20), longitude(28)) m s-1
}


In [28]:
surfacelayer=unifhycontrib.artemis.SurfaceLayerComponent(
        'outputs', timedomain, spacedomain, dataset_surfacelayer,
        parameters={})

model = build_model(surfacelayer, subsurface, openwater, nutrientsurfacelayer, nutrientsubsurface, nutrientopenwater)

KeyError: "no data 'air_temperature' available in DataSet for surfacelayer component 'SurfaceLayerComponent'"

A more helpful error message here would be "t2m" not found. "t2m" being the name of the variable that we specified was mapped to air_temperature.

If we replace "t2m" with the "long_name" attribute of the "t2m" variable in the NetCDF file, this error message goes away:

In [29]:
dataset_surfacelayer = unifhy.DataSet(
    files=['development/unifhy/data/surface_downwelling_longwave_flux_in_air.nc',
           'development/unifhy/data/surface_downwelling_shortwave_flux_in_air.nc',
           'development/unifhy/data/specific_humidity.nc',
           'unifhy_driving_files/era5-unedited/2m_temperature_2017.nc',
           'development/unifhy/data/wind_speed.nc',
           'development/unifhy/data/precipitation_flux.nc',
           'development/unifhy/data/leaf_area_index.nc',
           'development/unifhy/data/canopy_height.nc',
           'development/unifhy/data/soil_albedo.nc'],
    name_mapping={'leaf-area index': 'leaf_area_index',
                  'canopy height': 'vegetation_height',
                  'soil albedo': 'surface_albedo',
                  '2 metre temperature': 'air_temperature'} # do the name mapping here
)

print(dataset_surfacelayer)

DataSet{
    air_temperature(time(8760), latitude(47), longitude(44)) K
    leaf_area_index(time(12), latitude(360), longitude(720)) 1
    precipitation_flux(time(744), latitude(20), longitude(28)) kg m-2 s-1
    specific_humidity(time(744), latitude(20), longitude(28)) kg kg-1
    surface_albedo(latitude(360), longitude(720)) 1
    surface_downwelling_longwave_flux_in_air(time(744), latitude(20), longitude(28)) W m-2
    surface_downwelling_shortwave_flux_in_air(time(744), latitude(20), longitude(28)) W m-2
    vegetation_height(latitude(360), longitude(720)) m
    wind_speed(time(744), latitude(20), longitude(28)) m s-1
}


In [30]:
surfacelayer=unifhycontrib.artemis.SurfaceLayerComponent(
        'outputs', timedomain, spacedomain, dataset_surfacelayer,
        parameters={})

model = build_model(surfacelayer, subsurface, openwater, nutrientsurfacelayer, nutrientsubsurface, nutrientopenwater)

{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}
{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}
{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}
{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}
{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}


RuntimeError: field not compatible with LatLonGrid

But instead we get another error message, which I find is the most annoying one of all. 

**Problem 3: Unhelpful grid-incomptability error messages**

This error message doesn't tell you what exactly is wrong, as it as a 'catch-all' error message that is returned for any errors that occur in the spatial domain compatibility checks between the NetCDF files in the Dataset and the defined Space domain of the Component. Sometimes this a genuine incompatability error, where the NetCDF file doesn't cover the full Domain, or the resolutions mismatch, but it can also be very minor mismatches between the precise metadata of the NetCDF file's spatial domain and the spatial Domain of the Component., including missing metadata from the NetCDF file domain, which is very rarely supplied even on CF-Convention-compatible NetCDF files. This is what [Issue 54](https://github.com/unifhy-org/unifhy/issues/54) is getting at.

Let's see if we can find out what it is this time... ERA5 is a CF-Convention compatible dataset, so it *should* work out-of-the-box...

Inspecting the NetCDF file spatial domain:

In [31]:
!ncdump -h unifhy_driving_files/era5-unedited/2m_temperature_2017.nc | head -n 22 | tail -n 10

	float longitude(longitude) ;
		longitude:standard_name = "longitude" ;
		longitude:long_name = "longitude" ;
		longitude:units = "degrees_east" ;
		longitude:axis = "X" ;
	float latitude(latitude) ;
		latitude:standard_name = "latitude" ;
		latitude:long_name = "latitude" ;
		latitude:units = "degrees_north" ;
		latitude:axis = "Y" ;


In [32]:
!ncdump -v longitude unifhy_driving_files/era5-unedited/2m_temperature_2017.nc | tail -n 5

 longitude = -8.75, -8.5, -8.25, -8, -7.75, -7.5, -7.25, -7, -6.75, -6.5, 
    -6.25, -6, -5.75, -5.5, -5.25, -5, -4.75, -4.5, -4.25, -4, -3.75, -3.5, 
    -3.25, -3, -2.75, -2.5, -2.25, -2, -1.75, -1.5, -1.25, -1, -0.75, -0.5, 
    -0.25, 0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2 ;
}


In [33]:
!ncdump -v latitude unifhy_driving_files/era5-unedited/2m_temperature_2017.nc | tail -n 5

 latitude = 61, 60.75, 60.5, 60.25, 60, 59.75, 59.5, 59.25, 59, 58.75, 58.5, 
    58.25, 58, 57.75, 57.5, 57.25, 57, 56.75, 56.5, 56.25, 56, 55.75, 55.5, 
    55.25, 55, 54.75, 54.5, 54.25, 54, 53.75, 53.5, 53.25, 53, 52.75, 52.5, 
    52.25, 52, 51.75, 51.5, 51.25, 51, 50.75, 50.5, 50.25, 50, 49.75, 49.5 ;
}


Inspecting UniFHy's Space Domain:

In [34]:
print(spacedomain)

LatLonGrid(
    shape {Y, X}: (8, 6)
    Y, latitude (8,): [51.25, ..., 54.75] degrees_north
    X, longitude (6,): [-1.75, ..., 0.75] degrees_east
    Y_bounds (8, 2): [[51.0, ..., 55.0]] degrees_north
    X_bounds (6, 2): [[-2.0, ..., 1.0]] degrees_east
    cell_area (8, 6): [[1934901054.0364127, ..., 1784112017.7396371]] m2
    land_sea_mask (8, 6): [[True, ..., False]]
    flow_direction (8, 6, 2): [[[-1, ..., --]]]
)


In [35]:
for value in spacedomain.X:
    print(value)

-1.75 degrees_east
-1.25 degrees_east
-0.75 degrees_east
-0.25 degrees_east
0.25 degrees_east
0.75 degrees_east


In [36]:
for value in spacedomain.Y:
    print(value)

51.25 degrees_north
51.75 degrees_north
52.25 degrees_north
52.75 degrees_north
53.25 degrees_north
53.75 degrees_north
54.25 degrees_north
54.75 degrees_north


They have the same names (latitude and longitude), so that's fine.

The domain of the NetCDF file encloses that of UniFHy's Space Domain, so that's fine.

They are different resolutions, but multiples of each other (0.5 deg vs 0.25 deg), honestly can't remember if this is fine or not...

The NetCDF file lacks detailed spatial domain information. It could also be this.

The only way to find out for sure is to dig in to the UniFHy code and see where exactly the [```subset_and_compare```](https://github.com/unifhy-org/unifhy/blob/0fe448ffe7c5fb65be882cb6c8afb3a0dd9f6c39/unifhy/space.py#L2314) routine is raising an error.

It's in the [```is_space_equal_to```](https://github.com/unifhy-org/unifhy/blob/0fe448ffe7c5fb65be882cb6c8afb3a0dd9f6c39/unifhy/space.py#L2119) routine

This is the crucial bit of code from there, adapted to run here:

In [ ]:
#{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}
import numpy as np

kwargs = {'latitude': cf.Query('wi', [np.float64(51.25), np.float64(54.75)]),
          'longitude': cf.Query('wi', [np.float64(-1.75), np.float64(0.75)])}

datafield_subset = dataset_surfacelayer['air_temperature']._f.subspace(**kwargs)

unifhyspacedomain_lat = spacedomain._f.dim('latitude')
dataspacedomain_lat = datafield_subset.dim('latitude')

properties = list(dataspacedomain_lat.properties())
try:
    # these will be ignored except for 'units'
    properties.remove("units")
except ValueError:
    pass

unifhyspacedomain_lat.equals(
    dataspacedomain_lat,
    rtol=rtol_,
    atol=atol_,
    ignore_data_type=True,
    ignore_fill_value=True,
    ignore_properties=properties,
)

So CF-Python's ```equals``` function thinks the two space domains are different. Let's compare them by eye to see why.

In [39]:
print(unifhyspacedomain_lat)

latitude(8) degrees_north


In [40]:
print(dataspacedomain_lat)

latitude(15) degrees_north


So after a lot of digging, it looks like it's the resolution that's the issue this time.

If we fix that, does the problem go away?

In [77]:
dataset_surfacelayer = unifhy.DataSet(
    files=['development/unifhy/data/surface_downwelling_longwave_flux_in_air.nc',
           'development/unifhy/data/surface_downwelling_shortwave_flux_in_air.nc',
           'development/unifhy/data/specific_humidity.nc',
           'unifhy_driving_files/era5-resampled/2m_temperature_2017.nc',
           'development/unifhy/data/wind_speed.nc',
           'development/unifhy/data/precipitation_flux.nc',
           'development/unifhy/data/leaf_area_index.nc',
           'development/unifhy/data/canopy_height.nc',
           'development/unifhy/data/soil_albedo.nc'],
    name_mapping={'leaf-area index': 'leaf_area_index',
                  'canopy height': 'vegetation_height',
                  'soil albedo': 'surface_albedo',
                  '2 metre temperature': 'air_temperature'} # do the name mapping here
)

print(dataset_surfacelayer)

DataSet{
    air_temperature(time(8760), latitude(8), longitude(6)) K
    leaf_area_index(time(12), latitude(360), longitude(720)) 1
    precipitation_flux(time(744), latitude(20), longitude(28)) kg m-2 s-1
    specific_humidity(time(744), latitude(20), longitude(28)) kg kg-1
    surface_albedo(latitude(360), longitude(720)) 1
    surface_downwelling_longwave_flux_in_air(time(744), latitude(20), longitude(28)) W m-2
    surface_downwelling_shortwave_flux_in_air(time(744), latitude(20), longitude(28)) W m-2
    vegetation_height(latitude(360), longitude(720)) m
    wind_speed(time(744), latitude(20), longitude(28)) m s-1
}


In [106]:
#{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}
import numpy as np

kwargs = {'latitude': cf.Query('wi', [np.float64(51.25), np.float64(54.75)]),
          'longitude': cf.Query('wi', [np.float64(-1.75), np.float64(0.75)])}

datafield_subset = dataset_surfacelayer['air_temperature']._f.subspace(**kwargs)

unifhyspacedomain_lat = spacedomain._f.dim('latitude')
dataspacedomain_lat = datafield_subset.dim('latitude')

properties = list(dataspacedomain_lat.properties())
try:
    # these will be ignored except for 'units'
    properties.remove("units")
except ValueError:
    pass

unifhyspacedomain_lat.equals(
    dataspacedomain_lat,
    rtol=rtol_,
    atol=atol_,
    ignore_data_type=True,
    ignore_fill_value=True,
    ignore_properties=properties,
)

False

STILL FALSE!!!

In [79]:
print(unifhyspacedomain_lat)
print(dataspacedomain_lat)

latitude(8) degrees_north
latitude(8) degrees_north


In [81]:
for value in unifhyspacedomain_lat.data:
    print(value)

51.25 degrees_north
51.75 degrees_north
52.25 degrees_north
52.75 degrees_north
53.25 degrees_north
53.75 degrees_north
54.25 degrees_north
54.75 degrees_north


In [82]:
for value in dataspacedomain_lat.data:
    print(value)

51.25 degrees_north
51.75 degrees_north
52.25 degrees_north
52.75 degrees_north
53.25 degrees_north
53.75 degrees_north
54.25 degrees_north
54.75 degrees_north


The ERA5 data file and UniFHy's Space Domain are now exactly the same in terms of resolution and extent.

This is where the true problem lies. By most reasonable definitions these grids are now equal, and UniFHy should not be complaining. But it is, because it is looking at more than just the values, resolution and extent. 

In [107]:
unifhyspacedomain_lat.properties()

{'standard_name': 'latitude', 'units': 'degrees_north', 'axis': 'Y'}

I suspect this is where the problem is - the properties are not equal. 

If we make them equal does it pass?

In [114]:
#{'longitude': <CF Query: (wi [np.float64(-1.75), np.float64(0.75)])>, 'latitude': <CF Query: (wi [np.float64(51.25), np.float64(54.75)])>}
import numpy as np

kwargs = {'latitude': cf.Query('wi', [np.float64(51.25), np.float64(54.75)]),
          'longitude': cf.Query('wi', [np.float64(-1.75), np.float64(0.75)])}

datafield_subset = dataset_surfacelayer['air_temperature']._f.subspace(**kwargs)

unifhyspacedomain_lat = spacedomain._f.dim('latitude')
dataspacedomain_lat = datafield_subset.dim('latitude')

dataspacedomain_lat.del_property('_FillValue')
dataspacedomain_lat.del_property('long_name')

properties = list(dataspacedomain_lat.properties())
try:
    # these will be ignored except for 'units'
    properties.remove("units")
except ValueError:
    pass

unifhyspacedomain_lat.equals(
    dataspacedomain_lat,
    rtol=rtol_,
    atol=atol_,
    ignore_data_type=True,
    ignore_fill_value=True,
    ignore_properties=properties,
)

False

Somehow still no.

You see? This is what you have to do to get data to work with UniFHy. It's 6:20pm and I give up.